### Scrapeando la página Toc-Toc con Selenium
![Logo Toc-Toc](https://d1cfu8v5n1wsm.cloudfront.net/toctoc/img/logo-toctoc-200x200.jpg)

Utilizaremos Selenium para poder scrapear la página de *www.toctoc.com*, desde la cual, se obtendrán todos los detalles (valor, m2, nº de habitaciones, nº de baños, etc) de las propiedades usadas en venta(casas y departamentos) que se encuentren en la Región Metropolitana hasta el día de hoy.

### Selenium
Selenium es una herramienta que nos permitirá controlar un navegador y poder utilizar las funcionalidades del motor de JavaScript para cargar el contenido que no viene en el HTML de la página. Para esto, vamor a importar el módulo webdriver y el módulo keys (para poder indicar atajos del teclado para manipular el navegador remotamente)

In [12]:
from selenium import webdriver
#Para generar eventos de click
from selenium.webdriver.common.keys import Keys
#Para demoras dinámicas
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
#Para convertir listas a dataframe
import pandas as pd
#Para medir tiempo de ejecución de funciones
from time import time
from datetime import datetime
import sys

**Paso 1**: Instanciar un driver del navegador (en este caso, de Chrome). Asegurarse de que el archivo driver del navegador seleccionado se encuentre en la carpeta que se especifica en el path.

In [1207]:
options = webdriver.ChromeOptions() #Generamos una variable de opción vacía.
options.add_argument('--incognito') #le agregamos a la variable creada un argumento, en este caso de incógnito.
driver = webdriver.Chrome(executable_path= '/Users/danielparedesvillalobos/opt/anaconda3/chromedriver', options=options) #Le agregamos la variable creada al argumento options.

**Paso 2**: Generamos una variable con la url en cuestión y le inicamos al navegador que la carge (mediante webdriver)

In [1208]:
url_main = 'https://www.toctoc.com/resultados/lista/compra/casa-departamento/?moneda=2&precioDesde=0&precioHasta=0&dormitoriosDesde=&dormitoriosHasta=&banosDesde=0&banosHasta=0&estado=2&disponibilidadEntrega=&numeroDeDiasTocToc=0&superficieDesdeUtil=0&superficieHastaUtil=0&superficieDesdeConstruida=0&superficieHastaConstruida=0&superficieDesdeTerraza=0&superficieHastaTerraza=0&superficieDesdeTerreno=0&superficieHastaTerreno=0&ordenarPor=0&pagina=1&paginaInterna=1&zoom=15&idZonaHomogenea=0&atributos=&texto=Regi%C3%B3n%20Metropolitana%20De%20Santiago,%20Chile&viewport=-34.290931418749516,-71.83796149148834,-32.92240654382668,-69.64697082118785&idPoligono=2240&publicador=0&temporalidad=0'
driver.get(url_main)

**Paso 3**: Extraemos la información de interés.

In [1233]:
#Obtenemos los elementos de la lista de resultados (sólo la página actual)
propiedades = driver.find_elements_by_xpath('//li[@class="un-ress tp1"]')

In [1234]:
#Consultamos un sólo resultado de la lista para evaluar su composición
propiedad = propiedades[0]

In [1235]:
#Obtenemos la url de interés para el resultado que evaluamos (sólo el primero) y la abrimos en una nueva pestaña
pagina = propiedad.find_element_by_xpath('./a[@class="lnk-info"]')
pagina.send_keys(Keys.COMMAND + Keys.RETURN)



In [1236]:
# Hacemos foco sobre la nueva pestaña
driver.switch_to.window(driver.window_handles[1])

In [1246]:
# Cerrar la nueva pestaña de URL-secundaria
driver.close()

**Paso 3.1**: Generamos las funciones para realizar la extracción de datos

In [13]:
def obtener_info_propiedad(driver):
    '''
    Función que retorna un diccionario con las variables de interés de cada una de las propiedades de la página.
    Nota: El valor de la propiedad no se representará en la misma moneda por cada columna, 
    esto es algo que se debe limpiar después en la próxima etapa.
    '''
    #Obtenemos el código de la propiedad
    try:
        cod_propiedad = driver.find_element_by_xpath('//li[@class="cod"]').text
    except:
        cod_propiedad = ''
        
    #Obtenemos el tipo de propiedad
    try:
        if 'Departamento' in driver.find_element_by_xpath('//div[@class="info-cabecera-ficha"]').text:
            tipo_propiedad = 'Departamento'
        elif 'Casa' in driver.find_element_by_xpath('//div[@class="info-cabecera-ficha"]').text: 
            tipo_propiedad = 'Casa'
        else:
            tipo_propiedad = ''
    except:
        tipo_propiedad = ''
        
    #Obtenemos información general
    try:    
        info_general = driver.find_element_by_xpath('//div[@class="info-cabecera-ficha"]').text
    except:
        info_general = ''
        
    #Obtenemos el nombre principal
    try:    
        nombre_propiedad = driver.find_element_by_xpath('//h1[@class="tt-ficha"]').text
    except:
        nombre_propiedad =''
        
    #Obtenemos el precio A de la propiedad
    try:
        precio_a = driver.find_element_by_xpath('//div[@class="precio-b"]/strong').text
    except:
        precio_a=''
        
    #Obtenemos el precio B de la propiedad
    try:    
        precio_b = driver.find_element_by_xpath('//em[@class="precioAlternativo"]/strong').text
    except:
        precio_b =''
        
    #Obtenemos número de dormitorios
    try:
        n_dormitorios = driver.find_element_by_xpath('//*[@id="informacionBasica"]/div[2]/ul/li[2]/strong').text
    except:
        n_dormitorios =''
        
    #Obtenemos número de baños
    try:
        n_banos = driver.find_element_by_xpath('//li[@class="baños"]/strong').text[0]
    except:
        n_banos =''

    #Obtenemos superficie de terreno
    try:
        sup_terreno = driver.find_element_by_xpath('//li[@class="metrosTerreno"]/strong').text
    except:
        sup_terreno =''

    #Obtenemos superficie util
    try:
        sup_util = driver.find_element_by_xpath('//li[@class="metrosUtiles"]/strong').text
    except:
        sup_util = ''

    #Obtenemos superficie construida
    try:
        sup_construida = driver.find_element_by_xpath('//li[@class="metrosConstruidos"]/strong').text
    except:
        sup_construida =''

    #Obtenemos avaluo fiscal
    try:
        aval_fiscal = driver.find_element_by_xpath('//strong[@id="lblAvaluo"]').text
    except:
        aval_fiscal =''

    #Obtenemos fecha de publicación
    try:
        fecha_publicacion = driver.find_element_by_xpath('//*[@id="informacionBasica"]/div[2]/ul/li[16]/strong').text
    except:
        fecha_publicacion =''

    #Obtenemos última plusvalia del sector (fecha)
    try:
        fecha_plusvalia = driver.find_elements_by_xpath('//*[@id="chartPlusvalia"]/div/div[1]/div/div/table/tbody/tr/td[1]')[-1].get_attribute('innerHTML')
    except:
        fecha_plusvalia =''
        
    #Obtenemos última plusvalia del sector (valor)
    try:
        valor_plusvalia = driver.find_elements_by_xpath('//*[@id="chartPlusvalia"]/div/div[1]/div/div/table/tbody/tr/td[2]')[-1].get_attribute('innerHTML')
    except:
        valor_plusvalia =''
        
    #Obtenemos ubicación (latitud)
    try:
        latitud = driver.find_element_by_xpath('//input[@id="h_br_lat"]').get_attribute('value')
    except:
        latitud =''
        
    #Obtenemos ubicación (longitud)
    try:
        longitud = driver.find_element_by_xpath('//input[@id="h_br_lon"]').get_attribute('value')
    except:
        longitud =''
        
    info_propiedad = {'cod_propiedad':cod_propiedad,'tipo_propiedad':tipo_propiedad,'info_general':info_general,'nombre_propiedad':nombre_propiedad,
                      'precio_a':precio_a,'precio_b':precio_b,'n_dormitorios':n_dormitorios,'n_banos':n_banos,'sup_terreno':sup_terreno,
                      'sup_util':sup_util,'sup_construida':sup_construida,'aval_fiscal':aval_fiscal,'fecha_publicacion':fecha_publicacion,'fecha_plusvalia':fecha_plusvalia,'valor_plusvalia':valor_plusvalia,
                      'latitud':latitud,'longitud':longitud}
    print(f'   Se escrapea propiedad {nombre_propiedad}')
    return info_propiedad

In [25]:
def obtener_informacion(driver):
    '''
    Función que retorna una lista con todos los diccionarios de las variables obtenidas de cada una de las propiedades.
    '''
    hora_start = datetime.now().strftime('%Y-%m-%d %H:%M:%S') #Hora comienza función
    start_time = time() #Comienza conteo de tiempo
    print(f'El scraping se comienza a ejecutar a las {hora_start}')
    pag= int(driver.find_element_by_xpath('//li[@class="page-item active"]').text)
    prop = 16756 #Nº de propiedad con que comienza la descarga (sólo referencial, no condiciona la programación)
    info = []
    while driver.find_element_by_xpath('//a[@aria-label="Next"]'):
        try:
            propiedades = driver.find_elements_by_xpath('//li[@class="un-ress tp1"]')
            print(f'Se encontraron {len(propiedades)} propiedades en la página nº{pag}')#
            print(f'Inicializando scraping de la página nº{pag}...') #
            for propiedad in propiedades:
                propiedad.find_element_by_xpath('./a[@class="lnk-info"]').send_keys(Keys.COMMAND + Keys.RETURN)
                driver.switch_to.window(driver.window_handles[1])
                delay = 10                                                     
                try:
                    #Introducimos una demora dinámica
                    WebDriverWait(driver,delay).until(EC.presence_of_element_located((By.XPATH,'//*[@id="partialCabecera"]/div[1]/header/h1')))#//*[@id="hFicha"]#'/html/body/footer/div/div[1]/div[1]/h4'
                    #print(f'Inicializando carga de página propiedad nº{prop}')#
                    #print(f'    La página de la propiedad nº{prop} terminó de cargar')#
                    print(f'   Nº{prop}...inicializando scraping')                                    
                    try:
                        detalle_propiedad = obtener_info_propiedad(driver)
                    except:
                        print(f'    No fue posible scrapear la propiedad nº{prop}') 
                    prop +=1       
                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])

                except TimeoutException:
                    #print(f'Inicializando carga de página propiedad nº{prop}')#
                    print(f'Nº{prop}...la página de la propiedad tardó demasiado en cargar')
                    #print(f'    No fue posible scrapear la propiedad nº{prop}')#
                    #detalle_propiedad = []
                    prop +=1
                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])
                except:
                    #print(f'Inicializando carga de página propiedad nº{prop}')#
                    #print('     La página presenta algún problema')#
                    print(f'Nº{prop}...no fue posible scrapear la propiedad ') 
                    prop +=1
                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])
                info.append(detalle_propiedad)
            driver.find_element_by_xpath('//a[@aria-label="Next"]').click()
            pag +=1
            #if pag ==351:
                #break
            try:
                #Introducimos una demora dinámica para la nueva página
                WebDriverWait(driver,delay).until(EC.presence_of_element_located((By.XPATH,'//li[@class="page-item active"]')))
                print('La siguiente página terminó de cargar')
            except TimeoutException:
                print('La página tardó demasiado en cargar')
                return info
        except:
            print(f'El escraping se ha detenido inesperadamente')#,sys.exc_info()[0]
            
            hora_end = datetime.now().strftime('%Y-%m-%d %H:%M:%S') #Hora comienza función
            print(f'Se alcanzaron a escrapear {prop-1-16756} propiedades')
            print(f'Se retornarán los valores obtenidos hasta el momento')
            print(f'El scraping se comienza a ejecutar a las {hora_start}')
            print(f'El scraping se termina de ejecutar a las {hora_end}')
            elapsed_time = time() - start_time #Genera variable con la diferencia entre el comienzo y el término de la función
            print("Tiempo de ejecución: %.3f seconds." % elapsed_time)
            return info
        #finally:
             
    else:
        print("El scraping ha finalizado")
    
    print(f'Se escrapearon {prop-1-16756} propiedades')
    hora_end = datetime.now().strftime('%Y-%m-%d %H:%M:%S') #Hora comienza función
    print(f'El scraping se comienzó a ejecutar a las {hora_start}')
    print(f'El scraping se termina de ejecutar a las {hora_end}')
    elapsed_time = time() - start_time #Genera variable con la diferencia entre el comienzo y el término de la función
    print("Tiempo de ejecución: %.3f seconds." % elapsed_time)   
    return info
    

In [47]:
def obtener_informacion_arriendo(driver):
    '''
    Función que retorna una lista con todos los diccionarios de las variables obtenidas de cada una de las propiedades.
    '''
    hora_start = datetime.now().strftime('%Y-%m-%d %H:%M:%S') #Hora comienza función
    start_time = time() #Comienza conteo de tiempo
    print(f'El scraping se comienza a ejecutar a las {hora_start}')
    pag= int(driver.find_element_by_xpath('//li[@class="page-item active"]').text)
    prop = 1 #Nº de propiedad con que comienza la descarga (sólo referencial, no condiciona la programación)
    info = []
    while driver.find_element_by_xpath('//a[@aria-label="Next"]'):
        try:
            propiedades = driver.find_elements_by_xpath('//li[@class="un-ress tp3"]')
            print(f'Se encontraron {len(propiedades)} propiedades en la página nº{pag}')#
            print(f'Inicializando scraping de la página nº{pag}...') #
            for propiedad in propiedades:
                propiedad.find_element_by_xpath('./a[@class="lnk-info"]').send_keys(Keys.COMMAND + Keys.RETURN)
                driver.switch_to.window(driver.window_handles[1])
                delay = 10                                                     
                try:
                    #Introducimos una demora dinámica
                    WebDriverWait(driver,delay).until(EC.presence_of_element_located((By.XPATH,'//*[@id="partialCabecera"]/div[1]/header/h1')))#//*[@id="hFicha"]#'/html/body/footer/div/div[1]/div[1]/h4'
                    #print(f'Inicializando carga de página propiedad nº{prop}')#
                    #print(f'    La página de la propiedad nº{prop} terminó de cargar')#
                    print(f'   Nº{prop}...inicializando scraping')                                    
                    try:
                        detalle_propiedad = obtener_info_propiedad(driver)
                    except:
                        print(f'    No fue posible scrapear la propiedad nº{prop}') 
                    prop +=1       
                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])

                except TimeoutException:
                    #print(f'Inicializando carga de página propiedad nº{prop}')#
                    print(f'Nº{prop}...la página de la propiedad tardó demasiado en cargar')
                    #print(f'    No fue posible scrapear la propiedad nº{prop}')#
                    #detalle_propiedad = []
                    prop +=1
                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])
                except:
                    #print(f'Inicializando carga de página propiedad nº{prop}')#
                    #print('     La página presenta algún problema')#
                    print(f'Nº{prop}...no fue posible scrapear la propiedad ') 
                    prop +=1
                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])
                info.append(detalle_propiedad)
            driver.find_element_by_xpath('//a[@aria-label="Next"]').click()
            pag +=1
            if pag ==235:
                break
            try:
                #Introducimos una demora dinámica para la nueva página
                WebDriverWait(driver,delay).until(EC.presence_of_element_located((By.XPATH,'//li[@class="page-item active"]')))
                print('La siguiente página terminó de cargar')
            except TimeoutException:
                print('La página tardó demasiado en cargar')
                return info
            except:
                print('El scraping ha finalizado...')
        except:
            print(f'El escraping se ha detenido inesperadamente')#,sys.exc_info()[0]
            
            hora_end = datetime.now().strftime('%Y-%m-%d %H:%M:%S') #Hora comienza función
            print(f'Se alcanzaron a escrapear {prop-1} propiedades')
            print(f'Se retornarán los valores obtenidos hasta el momento')
            print(f'El scraping se comienza a ejecutar a las {hora_start}')
            print(f'El scraping se termina de ejecutar a las {hora_end}')
            elapsed_time = time() - start_time #Genera variable con la diferencia entre el comienzo y el término de la función
            print("Tiempo de ejecución: %.3f seconds." % elapsed_time)
            return info
        #finally:
             
    else:
        print("El scraping ha finalizado")
        
    
    print(f'Se escrapearon {prop-1} propiedades')
    hora_end = datetime.now().strftime('%Y-%m-%d %H:%M:%S') #Hora comienza función
    print(f'El scraping se comienzó a ejecutar a las {hora_start}')
    print(f'El scraping se termina de ejecutar a las {hora_end}')
    elapsed_time = time() - start_time #Genera variable con la diferencia entre el comienzo y el término de la función
    print("Tiempo de ejecución: %.3f seconds." % elapsed_time)   
    return info
    

In [65]:
options = webdriver.ChromeOptions() #Generamos una variable de opción vacía.
options.add_argument('--incognito') #le agregamos a la variable creada un argumento, en este caso de incógnito.
options.add_argument('--headless') #Para ejecutar en segundo plano
options.add_argument('--no-sandbox')#Para evitar crasheo de la página
options.add_argument('--disable-dev-shm-usage')#Para grandes cantidades de datos que requieren grandes cantidades de memoria (Mejor deshabilitar)
#options.add_argument('--window-size=WINDOW_SIZE') #Para manejar el tamaño de la ventana de chrome
driver = webdriver.Chrome(executable_path= '/Users/danielparedesvillalobos/opt/anaconda3/chromedriver', options=options) #Le agregamos la variable creada al argumento options.

In [6]:
#Para venta de propiedades
pag_inicial = 594
url_main = f'https://www.toctoc.com/resultados/lista/compra/casa-departamento/metropolitana/?moneda=2&precioDesde=0&precioHasta=0&dormitoriosDesde=&dormitoriosHasta=&banosDesde=0&banosHasta=0&estado=2&disponibilidadEntrega=&numeroDeDiasTocToc=0&superficieDesdeUtil=0&superficieHastaUtil=0&superficieDesdeConstruida=0&superficieHastaConstruida=0&superficieDesdeTerraza=0&superficieHastaTerraza=0&superficieDesdeTerreno=0&superficieHastaTerreno=0&ordenarPor=0&pagina=1&paginaInterna={pag_inicial}&zoom=15&idZonaHomogenea=0&atributos=&texto=Regi%C3%B3n%20Metropolitana%20De%20Santiago,%20Chile&viewport=-34.290931418749516,-71.83796149148834,-32.92240654382668,-69.64697082118785&idPoligono=2240&publicador=0&temporalidad=0#'
driver.get(url_main)

In [66]:
#Para arriendos de propiedades
pag_inicial = 31
url_arriendos = f'https://www.toctoc.com/resultados/lista/arriendo/casa-departamento/metropolitana/?moneda=2&precioDesde=0&precioHasta=0&dormitoriosDesde=&dormitoriosHasta=&banosDesde=0&banosHasta=0&estado=0&disponibilidadEntrega=&numeroDeDiasTocToc=0&superficieDesdeUtil=0&superficieHastaUtil=0&superficieDesdeConstruida=0&superficieHastaConstruida=0&superficieDesdeTerraza=0&superficieHastaTerraza=0&superficieDesdeTerreno=0&superficieHastaTerreno=0&ordenarPor=0&pagina=1&paginaInterna={pag_inicial}&zoom=15&idZonaHomogenea=0&atributos=&texto=Región%20Metropolitana%20De%20Santiago,%20Chile&viewport=-34.29093141875031,-71.83796149148732,-32.9224065438275,-69.64697082118687&idPoligono=2240&publicador=0&temporalidad=0#'
driver.get(url_arriendos)

In [45]:
#start_time = time() #Comienza conteo de tiempo
#seg_538_fin = obtener_informacion(driver)#desde página 538 y 15064 propiedades...tiempo indeterminado
#elapsed_time = time() - start_time #Genera variable con la diferencia entre el comienzo y el término de la función
#driver.close()
#print("Elapsed time: %.3f seconds." % elapsed_time)

In [ ]:
#Para arriendo
arr_seg_31_fin = obtener_informacion_arriendo(driver)
driver.close()

El scraping se comienza a ejecutar a las 2021-07-02 00:16:04
Se encontraron 30 propiedades en la página nº31
Inicializando scraping de la página nº31...
   Nº1...inicializando scraping
   Se escrapea propiedad :titulo:
   Nº2...inicializando scraping
   Se escrapea propiedad :titulo:
   Nº3...inicializando scraping
   Se escrapea propiedad Vitacura.
   Nº4...inicializando scraping
   Se escrapea propiedad Armando Jaramillo
   Nº5...inicializando scraping
   Se escrapea propiedad Nueva Costanera
   Nº6...inicializando scraping
   Se escrapea propiedad Alonso de Córdova
   Nº7...inicializando scraping
   Se escrapea propiedad Las Catalpas/Vitacura
   Nº8...inicializando scraping
   Se escrapea propiedad Las Nieves
   Nº9...inicializando scraping
   Se escrapea propiedad ALONSO DE CORDOVA - VITACURA/LAS NIEVES
   Nº10...inicializando scraping
   Se escrapea propiedad Departamento Amoblado Las Nieves
   Nº11...inicializando scraping
   Se escrapea propiedad Las Nieves /Las Catalpas
   Nº12

   Nº95...inicializando scraping
   Se escrapea propiedad Moderno y completo departamento / Metro I. de Suárez
   Nº96...inicializando scraping
   Se escrapea propiedad Barros Borgoño / Manuel Montt
   Nº97...inicializando scraping
   Se escrapea propiedad Eleodoro Yáñez / Antonio Varas
   Nº98...inicializando scraping
   Se escrapea propiedad ELIODORO YAÑEZ / FEDERICO FROEBEL
   Nº99...inicializando scraping
   Se escrapea propiedad HUERFANOS / AMUNÁTEGUI
   Nº100...inicializando scraping
   Se escrapea propiedad Metro La Moneda-Paseo Bulnes
   Nº101...inicializando scraping
   Se escrapea propiedad METRO SANTA ANA
   Nº102...inicializando scraping
   Se escrapea propiedad Av Manuel Rodríguez y Tucapel Jiménez/Moneda - Santiago Centro
   Nº103...inicializando scraping
   Se escrapea propiedad Eleuterio Ramírez con San Diego
   Nº104...inicializando scraping
   Se escrapea propiedad Lord Cochrane con Tarapacá
   Nº105...inicializando scraping
   Se escrapea propiedad Metro La Moneda
  

   Nº190...inicializando scraping
   Se escrapea propiedad Av. Kennedy, Gerónimo de Alderete
   Nº191...inicializando scraping
   Se escrapea propiedad CAMOENS 6470 / VITACURA
   Nº192...inicializando scraping
   Se escrapea propiedad av kennedy
   Nº193...inicializando scraping
   Se escrapea propiedad Kennedy lateral / Oscar Tenhann
   Nº194...inicializando scraping
   Se escrapea propiedad 10 de Julio Av Matta
   Nº195...inicializando scraping
   Se escrapea propiedad Metro Santa Isabel
   Nº196...inicializando scraping
   Se escrapea propiedad Se arrienda departamento cerca de todo
   Nº197...inicializando scraping
   Se escrapea propiedad Vicuña Mackenna
   Nº198...inicializando scraping
   Se escrapea propiedad Portugal / Copiapo
   Nº199...inicializando scraping
   Se escrapea propiedad Vicuña Mackenna/10 de Juliio
   Nº200...inicializando scraping
   Se escrapea propiedad Argomedo
   Nº201...inicializando scraping
   Se escrapea propiedad Fray Camilo Henríquez / Líder Express
 

   Nº285...inicializando scraping
   Se escrapea propiedad Portugal con Marín
   Nº286...inicializando scraping
   Se escrapea propiedad General Jofré con Fray Camilo Henriquez
   Nº287...inicializando scraping
   Se escrapea propiedad Metro Santa Isabel
   Nº288...inicializando scraping
   Se escrapea propiedad General Jofre - Vicuña Mackenna
   Nº289...inicializando scraping
   Se escrapea propiedad Lira
   Nº290...inicializando scraping
   Se escrapea propiedad Metro Parque Bustamante
   Nº291...inicializando scraping
   Se escrapea propiedad Santa Isabel / Lira
   Nº292...inicializando scraping
   Se escrapea propiedad Vicuña Mackenna entre Bilbao y Marín
   Nº293...inicializando scraping
   Se escrapea propiedad EXCELENTE CONECTIVIDAD
   Nº294...inicializando scraping
   Se escrapea propiedad :titulo:
   Nº295...inicializando scraping
   Se escrapea propiedad :titulo:
   Nº296...inicializando scraping
   Se escrapea propiedad Santa Isabel/Vicuña Mackenna
   Nº297...inicializando s

   Nº380...inicializando scraping
   Se escrapea propiedad SAN FRANCISCO DE ASIS / FRAY BERNARDO
   Nº381...inicializando scraping
   Se escrapea propiedad Las Condes - AMOBLADO
   Nº382...inicializando scraping
   Se escrapea propiedad CONDOMINIO. MIRADOR DE SAN DAMIAN
   Nº383...inicializando scraping
   Se escrapea propiedad San Carlos de Apoquindo.
   Nº384...inicializando scraping
   Se escrapea propiedad Casa en arriendo condominio exclusivo.( opción amoblada)
   Nº385...inicializando scraping
   Se escrapea propiedad GRAN VÍA / VÍA ROJA
   Nº386...inicializando scraping
   Se escrapea propiedad San Jorge - Francisco de Villagra
   Nº387...inicializando scraping
   Se escrapea propiedad Irarrázaval
   Nº388...inicializando scraping
   Se escrapea propiedad Plaza Egaña
   Nº389...inicializando scraping
   Se escrapea propiedad Irarrázaval / Heriberto Covarrubias
   Nº390...inicializando scraping
   Se escrapea propiedad Los Aliaga con Duble Almeyda
La siguiente página terminó de c

...

   Nº4569...inicializando scraping
   Se escrapea propiedad Capitan Crossbie
   Nº4570...inicializando scraping
   Se escrapea propiedad Wisconsin
   Nº4571...inicializando scraping
   Se escrapea propiedad Hermanos Cabot con las tranqueras
   Nº4572...inicializando scraping
   Se escrapea propiedad 
   Nº4573...inicializando scraping
   Se escrapea propiedad Santa Brígida/Noruega
   Nº4574...inicializando scraping
   Se escrapea propiedad Manquehue / Cerro El Plomo
   Nº4575...inicializando scraping
   Se escrapea propiedad San Olav
   Nº4576...inicializando scraping
   Se escrapea propiedad El Golf
   Nº4577...inicializando scraping
   Se escrapea propiedad Plaza El Golf
   Nº4578...inicializando scraping
   Se escrapea propiedad Clásico departamento amoblado en Sector el Golf*CON DOS ESTACIONAMIENTOS*
   Nº4579...inicializando scraping
   Se escrapea propiedad Napoleón
   Nº4580...inicializando scraping
   Se escrapea propiedad Exclusivo y seguro departamento Ideal para extranjeros 

Nº4665...la página de la propiedad tardó demasiado en cargar
   Nº4666...inicializando scraping
   Se escrapea propiedad Mall Plaza Egaña / Ñuñoa
   Nº4667...inicializando scraping
   Se escrapea propiedad Diagonal Oriente
   Nº4668...inicializando scraping
   Se escrapea propiedad Cauquenes
Nº4669...la página de la propiedad tardó demasiado en cargar
   Nº4670...inicializando scraping
   Se escrapea propiedad Fenelon
   Nº4671...inicializando scraping
   Se escrapea propiedad Av. La Dehesa/Berna
   Nº4672...inicializando scraping
   Se escrapea propiedad Berna
   Nº4673...inicializando scraping
   Se escrapea propiedad Universidad San Sebastián / Luis Thayer Ojeda
   Nº4674...inicializando scraping
   Se escrapea propiedad Los Leones / Lota
   Nº4675...inicializando scraping
   Se escrapea propiedad AMOBLADO, A PASOS DEL METRO
   Nº4676...inicializando scraping
   Se escrapea propiedad Bosque Norte - Las Hortensias/ Los Leones
   Nº4677...inicializando scraping
   Se escrapea propieda

   Nº4762...inicializando scraping
   Se escrapea propiedad SANTA ELENA 1298 DP 910, Santiago
Nº4763...la página de la propiedad tardó demasiado en cargar
   Nº4764...inicializando scraping
   Se escrapea propiedad CARMEN 1153 EDIF. 1 DP 2306 DP 2306, Santiago
   Nº4765...inicializando scraping
   Se escrapea propiedad Fray Camilo Henríquez
   Nº4766...inicializando scraping
   Se escrapea propiedad MUJICA 55 DP 1411 DP 1411, Ñuñoa
Nº4767...la página de la propiedad tardó demasiado en cargar
La siguiente página terminó de cargar
Se encontraron 30 propiedades en la página nº190
Inicializando scraping de la página nº190...
   Nº4768...inicializando scraping
   Se escrapea propiedad Avenida Portugal
Nº4769...la página de la propiedad tardó demasiado en cargar
   Nº4770...inicializando scraping
   Se escrapea propiedad Fray Camilo Henríquez
   Nº4771...inicializando scraping
   Se escrapea propiedad Sara del Campo
   Nº4772...inicializando scraping
   Se escrapea propiedad Departamento 2D2

   Nº4857...inicializando scraping
   Se escrapea propiedad Matucana
La siguiente página terminó de cargar
Se encontraron 30 propiedades en la página nº193
Inicializando scraping de la página nº193...
   Nº4858...inicializando scraping
   Se escrapea propiedad Matucana
   Nº4859...inicializando scraping
   Se escrapea propiedad Matucana
   Nº4860...inicializando scraping
   Se escrapea propiedad Rivas Vicuña
   Nº4861...inicializando scraping
   Se escrapea propiedad Matucana
   Nº4862...inicializando scraping
   Se escrapea propiedad Matucana
   Nº4863...inicializando scraping
   Se escrapea propiedad Matucana
   Nº4864...inicializando scraping
   Se escrapea propiedad Matucana
   Nº4865...inicializando scraping
   Se escrapea propiedad Matucana
   Nº4866...inicializando scraping
   Se escrapea propiedad Matucana
   Nº4867...inicializando scraping
   Se escrapea propiedad Matucana
   Nº4868...inicializando scraping
   Se escrapea propiedad Calle Uno
   Nº4869...inicializando scraping


Nº4952...la página de la propiedad tardó demasiado en cargar
Nº4953...la página de la propiedad tardó demasiado en cargar
   Nº4954...inicializando scraping
   Se escrapea propiedad El Molino
   Nº4955...inicializando scraping
   Se escrapea propiedad Ortúzar
   Nº4956...inicializando scraping
   Se escrapea propiedad Alcalde Jorge Monckeberg 35, Ñuñoa
   Nº4957...inicializando scraping
   Se escrapea propiedad Capitán Orella
   Nº4958...inicializando scraping
   Se escrapea propiedad Los tres Antonios 377, Ñuñoa
   Nº4959...inicializando scraping
   Se escrapea propiedad Armando Jaramillo
   Nº4960...inicializando scraping
   Se escrapea propiedad Magdalena
Nº4961...la página de la propiedad tardó demasiado en cargar
   Nº4962...inicializando scraping
   Se escrapea propiedad Metro El Golf - 60 mts2
   Nº4963...inicializando scraping
   Se escrapea propiedad Frankfort
   Nº4964...inicializando scraping
   Se escrapea propiedad Vicuña Mackenna
   Nº4965...inicializando scraping
   Se e

In [52]:
arr_seg_1_30 = arr_seg_1_fin

In [8]:
seg_538_593

NameError: name 'seg_538_593' is not defined

**Paso 4**: Se genera un data frame único con cada uno de los segmentos de datos obtenidos.

In [58]:
#Para venta
seg_1_35 #Desde la página 1:35, 1050 propiedades (1050 acumuladas)
seg_36_343 #Desde la página 36:343, 9247 propiedades (10297 acumuladas)
seg_344_416 #Desde la página 344:416, 2190 propiedades (11437 acumuladas)
seg_417_537 #Desde la página 417:537, 3627 propiedades (15064 acumuladas)
seg_538_593 #Desde la página 538:593, 1692 propiedades (16756 acumuladas)

In [1535]:
#Para arriendos
arr_seg_1_30 #Desde la página 1:30, 935 propiedades (935 acumuladas)

In [60]:
df = pd.DataFrame(arr_seg_1_30)
#df

In [56]:
df.to_csv('data_muestra_arriendo_propiedades_toctoc_1.csv')

In [1409]:
# Hacemos foco sobre la pestaña principal
driver.switch_to.window(driver.window_handles[0])

In [1039]:
#Crear variable que permita pasar de página con el evento click
pasar_pagina = driver.find_element_by_xpath('//a[@aria-label="Next"]')
pasar_pagina.click()

**Paso 4**: Cerramos el navegador.

In [1724]:
driver.close()